In [1]:
import pandas as pd 
import numpy as np 
import infrastructure
import filepath 
import pygsheets
import datetime as dt
import openpyxl 
import os
from pathlib import Path

In [2]:
import xlsxwriter
import send_email

# IMPORT DATA

In [3]:
#manual input
df = pd.read_csv('SS_LC_merged_data.csv')

/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,2,3,4,6,8,9,10,11,12,14,16,26,27,28,29,30,31,32,33,34,35,36,37,44,48,51,54,57,58,62) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df.columns

Index(['Date', 'Scheduling Time', 'Offer', 'Hitpath_Offer_ID', 'DP.SV',
       'Affiliate_Id', 'DP&Pub', 'Job_Id', 'Job_Name', 'Creative_Id',
       'Creativename', 'Creative', 'Send Strategy', 'Shortcode',
       'Start_Tstamp', 'Sent', 'Segments', 'Revenue', 'Conversions',
       'Jump Page Clicks', 'Delivered', 'Not_Delivered', 'Optout', 'Clicks',
       'Cost', 'Ecpm', 'Time', 'Publisher', 'Campaign', 'Route', 'Carrier',
       'Dataset', 'Message', 'Responder Template', 'Keyword', 'Responder',
       'Router Domain Name', 'c1', 'Responded', 'Response Rate', 'CTR',
       'Gross Profit', 'Gross Margin', 'RPU', 'Provider', 'Code_Type',
       'Revenue Source', 'Ar Day', 'Sub_Id', 'Campaign_Id', 'Roi',
       'Shortcode Name', 'Total', 'Jump Page Version', 'Offer Wall Id',
       'Offer Type', 'campaign_id', 'campaign_name', 'shortcode_DP.SV',
       'Opportunity Cost Send Strategy', 'Dataset_Agg_30D_eCPM',
       'Opportunity Cost', 'Offer Vertical', 'Offer Gap', 'Vertical Gap',
   

In [5]:
# SMS testing pipeline
sms_test = infrastructure.get_smartsheet('test_sms')
sms_test['Hitpath Offer ID'] = sms_test['Hitpath Offer ID'].astype(str)
# SMS offer sheet 
sms_offer = infrastructure.get_smartsheet('offers_sms')
sms_offer = sms_offer[sms_offer['Hitpath Offer ID'].isna() == False]

#SMS Offer wall
sms_ow = infrastructure.get_smartsheet('ow_sms')
# sms_offer = sms_offer[sms_offer['Hitpath ID'].str.contains("TEST") == False]
# sms_offer = sms_offer[~sms_offer['Hitpath ID'].str.contains(r'test\d*', regex=True)]
sms_offer['Hitpath Offer ID'] = sms_offer['Hitpath Offer ID'].astype(int).astype(str)
# Email offer sheet 
email_offer = infrastructure.get_smartsheet('offers_email')
# get mamba for shortcode 
mamba = infrastructure.get_mamba()
# Offer Testing Timeline 
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
timeline = gc.open_by_url('https://docs.google.com/spreadsheets/d/16vrHMWs0ambcBJ1sC0SqpYVG8SOSWlbg1N22-bF49v8/edit#gid=423634805') 
timeline_wks  = timeline[1].get_as_df()

/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/pygsheets/worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')
/Users/nathan/Documents/SMS DMA Python/SMS DataSource/infrastructure.py:112: FutureWarning: The default value of regex will change from True to False in a future version.
  snakes['Hitpath Offer ID']=snakes['Hitpath Offer ID'].astype(str).str.replace('.0', '')


In [6]:
sms_offer.columns

Index(['Hitpath Offer ID', 'Email Hitpath Offer ID', 'Scheduling Name',
       'Offer Name', 'Status', 'Status Changed Date', 'Delayed Reporting',
       'RX Rep', 'Advertiser', 'Data Partner Relationship',
       'Advertiser Offer ID', 'Direct Traffic Accepted ',
       'Additional Notes/Pre-pop Instructions', 'Vertical', 'Redirect Link',
       'Custom Creative Allowed', 'Custom Content Status',
       'Adv/Offer Abbreviation', 'Advertiser Rep', 'Scheduled Test Date',
       'Day Restrictions', 'Other Restrictions', 'Payout Change',
       'Payout Change Effective Date', '$ Payout', 'Payout Type',
       'Budget Status', 'Budget', 'Cap', 'Portal Acct Info', 'Live Pub IDs',
       'Paused Pubs', 'Special Pub Payouts', 'Pitch Approval',
       'Single/Multiple Pitch'],
      dtype='object')

In [7]:
df[df['Hitpath_Offer_ID'] == 'OW13']

,Date,Scheduling Time,Offer,Hitpath_Offer_ID,DP.SV,Affiliate_Id,DP&Pub,Job_Id,Job_Name,Creative_Id,...,campaign_id,campaign_name,shortcode_DP.SV,Opportunity Cost Send Strategy,Dataset_Agg_30D_eCPM,Opportunity Cost,Offer Vertical,Offer Gap,Vertical Gap,eCPM ratio
151667,2024-05-11,2024-05-11 12:00,OW13 - GF HZB,OW13,TLG.PL,461768.0,TLG.PL_461768,663307.0,SS_HZB_TLG-PL-30DC-VZN_OW13_P_11May24,852176.0,...,NaN,NaN,HZB_TLG.PL,True,19.480058,-64.017595,NaN,14.0,NaN,0.194925
241344,2024-05-07,2024-05-07 08:30,OW13 - GF MBC,OW13,WWM.YFA.2,461680.0,WWM.YFA.2_461680,662115.0,SS_MBC_WWM-YFA-2-1PLD-15DC-TMO_OW13_OT_07May24,815823.0,...,NaN,NaN,MBC_WWM.YFA.2,True,19.309569,-1.294920,NaN,NaN,NaN,0.794291
286962,2024-05-02,2024-05-02 10:00,OW13 - GF DSS,OW13,TLG.PL,461768.0,TLG.PL_461768,660528.0,SS_DSS_TLG-PL-15DC_OW13_OT_02May24,835609.0,...,NaN,NaN,DSS_TLG.PL,True,17.101387,-26.617268,NaN,NaN,NaN,0.219830
340393,2024-04-27,2024-04-27 12:00,OW13 - GF HZB,OW13,TLG.PL,461768.0,TLG.PL_461768,659082.0,SS_HZB_TLG-PL-30DC-VZN_OW13_P_27Apr24,852176.0,...,NaN,NaN,HZB_TLG.PL,True,22.205184,4.157061,NaN,35.0,NaN,1.046791
407391,2024-04-21,2024-04-21 12:00,OW13 - GF HZB,OW13,DOT.PL,461039.0,DOT.PL_461039,657449.0,SS_HZB_DOT-PL-30DC-VZN_OW13_P_21Apr24,852176.0,...,NaN,NaN,HZB_DOT.PL,True,12.833832,-10.693179,NaN,NaN,NaN,0.189492
525520,2024-04-07,2024-04-07 10:00,OW13 - GF FLC,OW13,JET.ZTA,461835.0,JET.ZTA_461835,653442.0,SS_FLC_JET-ZTA-30DC-VZN_OW13_P_07Apr24,835607.0,...,NaN,NaN,FLC_JET.ZTA,True,10.431984,2.090947,NaN,3.0,NaN,2.095280
556292,2024-04-04,2024-04-04 10:00,OW13 - GF FLC,OW13,JET.ZTA,461835.0,JET.ZTA_461835,652762.0,SS_FLC_JET-ZTA-30DC-VZN_OW13_P_04Apr24,835607.0,...,NaN,NaN,FLC_JET.ZTA,True,9.970783,1.095580,NaN,4.0,NaN,1.575283
580407,2024-03-31,2024-03-31 10:00,OW13 - GF FLC,OW13,JET.ZTA,461835.0,JET.ZTA_461835,651470.0,SS_FLC_JET-ZTA-30DC-VZN_OW13_P_31Mar24,835607.0,...,NaN,NaN,FLC_JET.ZTA,True,9.261452,0.221801,NaN,15.0,NaN,1.124734
586382,2024-03-23,2024-03-23 12:00,OW13 - GF HZB,OW13,TLG.PL,461768.0,TLG.PL_461768,649027.0,SS_HZB_TLG-PL-30DC-VZN_OW13_P_23Mar24,852176.0,...,NaN,NaN,HZB_TLG.PL,True,41.141088,-21.498744,NaN,NaN,NaN,0.494132
586565,2024-03-23,2024-03-23 10:00,OW13 - GF FLC,OW13,PN.FC,461653.0,PN.FC_461653,649053.0,SS_FLC_PN-FC-30DC-VZN_OW13_P_23Mar24,835607.0,...,NaN,NaN,FLC_PN.FC,True,19.686351,-53.519509,NaN,7.0,NaN,0.536706


In [8]:
df.columns

Index(['Date', 'Scheduling Time', 'Offer', 'Hitpath_Offer_ID', 'DP.SV',
       'Affiliate_Id', 'DP&Pub', 'Job_Id', 'Job_Name', 'Creative_Id',
       'Creativename', 'Creative', 'Send Strategy', 'Shortcode',
       'Start_Tstamp', 'Sent', 'Segments', 'Revenue', 'Conversions',
       'Jump Page Clicks', 'Delivered', 'Not_Delivered', 'Optout', 'Clicks',
       'Cost', 'Ecpm', 'Time', 'Publisher', 'Campaign', 'Route', 'Carrier',
       'Dataset', 'Message', 'Responder Template', 'Keyword', 'Responder',
       'Router Domain Name', 'c1', 'Responded', 'Response Rate', 'CTR',
       'Gross Profit', 'Gross Margin', 'RPU', 'Provider', 'Code_Type',
       'Revenue Source', 'Ar Day', 'Sub_Id', 'Campaign_Id', 'Roi',
       'Shortcode Name', 'Total', 'Jump Page Version', 'Offer Wall Id',
       'Offer Type', 'campaign_id', 'campaign_name', 'shortcode_DP.SV',
       'Opportunity Cost Send Strategy', 'Dataset_Agg_30D_eCPM',
       'Opportunity Cost', 'Offer Vertical', 'Offer Gap', 'Vertical Gap',
   

In [9]:
#Publisher Config
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
publisher_config = gc.open_by_url(
    'https://docs.google.com/spreadsheets/d/1Tzda6Djr3zQmOhWu7Ief3GVR9Cjaml8238CeX7chj_U/edit#gid=1620368362') 
pub_config = publisher_config[0].get_as_df()
pub_config.rename(columns={'DP.DS or DP.sV': 'DP.SV'}, inplace=True)

/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/pygsheets/worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')


In [10]:
#la nina
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
lanina = gc.open_by_url('https://docs.google.com/spreadsheets/d/1obszkCQoE0ELOR1O0CrLVETUEmEIWlGuyAmK3FgWSJg/edit#gid=1099746391') 
lanina_wks  = lanina[1].get_as_df()

In [11]:
#sms offer restrictions
sms_restrictions = sms_offer[['Hitpath Offer ID', 'Paused Pubs']].copy()

In [12]:
#rename and retype
df.rename(columns={'Hitpath_Offer_ID': 'Hitpath Offer ID'}, inplace=True)
df['Hitpath Offer ID'] = pd.to_numeric(df['Hitpath Offer ID'], errors='coerce')
df = df.dropna(subset=['Hitpath Offer ID'])
df['Hitpath Offer ID'] = df['Hitpath Offer ID'].astype(int).astype(str)

df['Affiliate_Id'] = pd.to_numeric(df['Affiliate_Id'], errors='coerce')
df = df.dropna(subset=['Affiliate_Id'])
df['Affiliate_Id'] = df['Affiliate_Id'].astype(int).astype(str)


In [13]:
df = df[df['Send Strategy'] != 'AR']

In [14]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [15]:
#mamba schedule from today and onward
today = pd.to_datetime(dt.date.today())

mamba_future = mamba[mamba['Date'] >= today]

mamba_future

,Date,Dataset,Drop,Time,Segment,Send Strategy,Offer,Limit,Offset,Creative,Job Name,Hitpath Offer ID,Shortcode,shortcode_DP.SV
23612,2024-05-16,DSS_IM.SVY_461916,Drop 2,10:00 AM PST,DSS_IM.SVY_30DC,OT,11646 - B2 CGC DSS,2300,0,CGC.11646.SC.DSS.451492,SS_DSS_IM-SVY-30DC_11646_OT_16May24,11646,DSS,DSS_IM.SVY
23617,2024-05-16,DSS_JET.ZTA_461835,Drop 1,10:00 AM PST,DSS_JET.ZTA_1PLD.15DC,P,13253 - DSS,,,13253.SC.DSS.454051,SS_DSS_JET-ZTA-1PLD-15DC_13253_P_16May24,13253,DSS,DSS_JET.ZTA
23629,2024-05-16,DSS_ZM.PL.2_461681,Drop 1,10:00 AM PST,DSS_ZM.PL.2_15DC,OT,13253 - DSS,,,13253.SC.DSS.454051,SS_DSS_ZM-PL-2-15DC_13253_OT_16May24,13253,DSS,DSS_ZM.PL.2
23635,2024-05-16,FLC_EDM.247L_461227,Drop 1,10:00 AM PST,FLC_EDM.247L_30DC_NEXL,P,13221 - FLC,Total - 2101,2101,13221.SC.FLC.454071,SS_FLC_EDM-247L-30DC-NEXL_13221_P_16May24,13221,FLC,FLC_EDM.247L
23636,2024-05-16,FLC_EDM.247L_461227,Drop 2,10:00 AM PST,FLC_EDM.247L_30DC_NEXL,OT,13089 - FLC,2100,0,13089.SC.FLC.453971,SS_FLC_EDM-247L-30DC-NEXL_13089_OT_16May24,189,FLC,FLC_EDM.247L
23637,2024-05-16,FLC_EDM.247L_461227,Drop 3,10:00 AM PST,FLC_EDM.247L_30DC_EXL,P,13221 - FLC,,,13221.SC.FLC.454071,SS_FLC_EDM-247L-30DC-EXL_13221_P_16May24,13221,FLC,FLC_EDM.247L
23638,2024-05-16,FLC_EDM.247L_461227,Drop 4,10:00 AM PST,FLC_EDM.247L_30DC_NT,P,13221 - FLC,,,13221.SC.FLC.454071,SS_FLC_EDM-247L-30DC-NT_13221_P_16May24,13221,FLC,FLC_EDM.247L
23647,2024-05-16,FLC_I.CC_460918,Drop 1,10:00 AM PST,FLC_I.CC_30DC,P,13113 - FLC,,,13113.SC.FLC.453641,SS_FLC_I-CC-30DC_13113_P_16May24,13113,FLC,FLC_I.CC
23659,2024-05-16,FLC_PN.FC_461653,Drop 1,10:00 AM PST,FLC_PN.FC_30DC_VZN,P,13253 - FLC,,,13253.SC.FLC.454051,SS_FLC_PN-FC-30DC-VZN_13253_P_16May24,13253,FLC,FLC_PN.FC
23665,2024-05-16,FLC_SPK.CR2_461842,Drop 1,10:00 AM PST,FLC_SPK.CR2_30DC_VZN,P,12972 - FLC,,,12972.SC.FLC.453371,SS_FLC_SPK-CR2-30DC-VZN_12972_P_16May24,12972,FLC,FLC_SPK.CR2


# Most Recent Passed Offers

In [16]:
sms_offer

,Hitpath Offer ID,Email Hitpath Offer ID,Scheduling Name,Offer Name,Status,Status Changed Date,Delayed Reporting,RX Rep,Advertiser,Data Partner Relationship,Advertiser Offer ID,Direct Traffic Accepted,Additional Notes/Pre-pop Instructions,Vertical,Redirect Link,Custom Creative Allowed,Custom Content Status,Adv/Offer Abbreviation,Advertiser Rep,Scheduled Test Date,Day Restrictions,Other Restrictions,Payout Change,Payout Change Effective Date,$ Payout,Payout Type,Budget Status,Budget,Cap,Portal Acct Info,Live Pub IDs,Paused Pubs,Special Pub Payouts,Pitch Approval,Single/Multiple Pitch
4,13152,12980.0,13152-Credit Card-3 West Media-Indigo Masterca...,Indigo Mastercard CPL v3 (non-rewards),Set Up: Pending Custom Content,03/25/24,NaN,Sarah Bowe,3 West Media,NaN,Di,NaN,- Creative Footer: Must include the unique tra...,Credit Card,https://www.spectrumlinkx.com/nOQlUFhksNGUm8n1...,Yes - need advertiser approval,Pending Approval - custom content has been gen...,NaN,Andrew Bukovics,NaN,NaN,Traffic is allowed from USA ONLY.,NaN,NaN,$52.00,CPL,Non Budgeted,NaN,NaN,3 West Media(ID:28528) - Haley - SMS,NaN,NaN,NaN,NaN,Single
5,12920,11206.0,12920-Credit Card-3 West Media-Milestone Maste...,Milestone Mastercard - Cashback Rewards CPL (NF),Set Up: Pending Custom Content,01/24/24,NaN,Sarah Bowe,3 West Media,NaN,733,NaN,NaN,Credit Card,https://www.spectrumlinkx.com/nOQlUFhksNGUm8n1...,Yes - need advertiser approval,Pending Approval - custom content has been gen...,3WM_MCB,Patrick Welty,Not in Testing Pipeline,NaN,NaN,(+) $12\n(-) $8,04/04/52,52,CPL,NaN,NaN,NaN,28528,NaN,NaN,NaN,Approved,Single
6,12919,10914.0,12919-Credit Card-3 West Media-Destiny Masterc...,Destiny Mastercard - Cashback Rewards CPL (NF),Set Up: Pending Custom Content,01/24/24,NaN,Sarah Bowe,3 West Media,NaN,731,NaN,NaN,Credit Card,https://www.spectrumlinkx.com/nOQlUFhksNGUm8n1...,Yes - need advertiser approval,Pending Approval - custom content has been gen...,3WM_DCB,Patrick Welty,Not in Testing Pipeline,NaN,NaN,(+) $12\n(-) $8,02/19/24,52,CPL,NaN,NaN,NaN,28528,NaN,NaN,NaN,Approved,Single
7,12917,12396.0,12917-Credit Card-3 West Media-Destiny Masterc...,Destiny Mastercard - Instant Credit CPL,Set Up: Pending Custom Content,01/24/24,NaN,Sarah Bowe,3 West Media,NaN,797,NaN,NaN,Credit Card,https://www.spectrumlinkx.com/nOQlUFhksNGUm8n1...,Yes - need advertiser approval,Pending Approval - custom content has been gen...,3WM_DI,Patrick Welty,Not in Testing Pipeline,NaN,NaN,NaN,NaN,50,CPL,NaN,NaN,NaN,28528,NaN,NaN,NaN,Approved,Single
8,12916,NaN,12916-Auto Warranty-3 West Media-Road Master A...,Road Master Auto Warranty CPL,Set Up: Pending Custom Content,01/24/24,NaN,Sarah Bowe,3 West Media,NaN,833,NaN,NaN,Auto Warranty,https://www.spectrumlinkx.com/nOQlUFhksNGUm8n1...,Yes - need advertiser approval,Pending Approval - custom content has been gen...,3WM_RM,Patrick Welty,Not in Testing Pipeline,NaN,NaN,NaN,NaN,16,CPL,NaN,NaN,NaN,28528,NaN,NaN,NaN,Approved,Single
9,11890,11206.0,11890-Credit Card-3 West Media-Milestone Maste...,Milestone Mastercard - Cashback Rewards CPL DO...,Live,NaN,NaN,Sarah Bowe,3 West Media,NaN,NaN,NaN,NaN,Credit Card,https://www.milestonegoldcard.com/apply-form?c...,No - the campaign does NOT accept custom content,NaN,3W_MM,NaN,05/10/23,NaN,COPY EMAIL CAMPAIGN ID 11206- Milestone Master...,(+) $12\n(-) $8,02/19/24,52,CPA,NaN,NaN,No Cap,NaN,NaN,NaN,NaN,NaN,NaN
10,12066,11914.0,12066-Credit Card-3 West Media-Credit Card-3 W...,Credit Card-3 West Media-Go2Bank Credit Card C...,Paused: Failed Testing,NaN,NaN,Sarah Bowe,3 West Media,NaN,NaN,NaN,NaN,Credit Card,https://www.go2bank.com/?gdclkid=xatRQISBbxyNW...,No - the campaign does NOT accept custom content,NaN,3W_G2B,NaN,07/03/23,NaN,*COPY EMAIL CAMPAIGN*,$21.00,NaN,$17.00,CPL,NaN,"$13,500.00",Monthly,NaN,NaN,NaN,NaN,NaN,NaN
11,12456,12301.0,12456-Credit Card-3 West Media-Indigo Masterca...,Indigo Mastercard DO NOT CONTACT (SMS- Email O...,Paused: All Else,NaN,NaN,Sarah Bowe,3 West Media,NaN,NaN,NaN,NaN,Credit Card,h

In [17]:
sms_offer['Status Changed Date'] = pd.to_datetime(sms_offer['Status Changed Date'], errors='coerce')

#get the current date to calculate the last 30 days range
current_date = pd.to_datetime('now').normalize()

sms_offer_filtered = sms_offer[
    ((sms_offer['Status'] == 'Live') | (sms_offer['Status'] == 'Live - Budgeted Offer')) &
    (sms_offer['Status Changed Date'] >= current_date - pd.Timedelta(days=30))
]

sms_offer_filtered.reset_index(drop=True, inplace=True)
sms_offer_filtered = sms_offer_filtered[
    ['Hitpath Offer ID', 'Scheduling Name', 'Advertiser', '$ Payout', 'Payout Type', 'Vertical', 'Day Restrictions']
]

In [18]:
sms_ow['Scheduled Test Date'] = pd.to_datetime(sms_ow['Scheduled Test Date'], errors='coerce')

sms_ow_filtered = sms_ow[
    (sms_ow['Status'] == 'Live')  & 
    (sms_ow['Scheduled Test Date'] >= current_date - pd.Timedelta(days=30))]

sms_ow_filtered.reset_index(drop=True, inplace=True)
sms_ow_filtered = sms_ow_filtered[['Hitpath Offer ID', 'Vertical']]

In [19]:
sms_offer_filtered = pd.concat([sms_offer_filtered, sms_ow_filtered], sort=False)

In [20]:
#calculating eCPM for passed offers from PT performance
df_pt = df[df['Send Strategy'] == 'PT']
merged_df = pd.merge(sms_offer_filtered, df_pt, on='Hitpath Offer ID', how='left')

# Grouping by 'Hitpath ID' and calculating total revenue and total delivered for each
grouped_stats = merged_df.groupby('Hitpath Offer ID').agg({'Revenue': 'sum', 'Delivered': 'sum'}).reset_index()
passed_offers = pd.merge(sms_offer_filtered, grouped_stats, on='Hitpath Offer ID', how='left')
passed_offers['eCPM'] = passed_offers['Revenue']/passed_offers['Delivered']*1000

In [21]:
#vertical rec
df_vertical = df.merge(pub_config[['DP.SV', 'Sub Vertical', 'INTERNAL STATUS']], on='DP.SV', how='left')
df_vertical = df_vertical.merge(sms_offer[['Hitpath Offer ID', 'Vertical', 'Paused Pubs', 'Status']], on='Hitpath Offer ID', how='left')

In [22]:
grouped_data = df_vertical.groupby(['Vertical', 'Sub Vertical']).agg({'Revenue': 'sum', 'Delivered': 'sum'}).reset_index()

# eCPM for grouped data
grouped_data['eCPM'] = (grouped_data['Revenue']/grouped_data['Delivered'])*1000

# Filtering for combinations where the calculated eCPM is greater than $16
filtered_combinations = grouped_data[grouped_data['eCPM'] > 12]

# Getting all unique sub-verticals with eCPM > $16 for each vertical
sub_verticals_eCPM_over_16 = filtered_combinations.groupby('Vertical')['Sub Vertical'].unique()

# Mapping these sub-verticals back to the 'passed offers' dataframe
passed_offers['Data Vertical Recommendations'] = passed_offers['Vertical'].map(sub_verticals_eCPM_over_16)

In [23]:
passed_offers

,Hitpath Offer ID,Scheduling Name,Advertiser,$ Payout,Payout Type,Vertical,Day Restrictions,Revenue,Delivered,eCPM,Data Vertical Recommendations
0,13238,13238-Home Warranty-Admediary-Total Home Guard...,Admediary,$30.00,CPL,Home Warranty,Monday - Friday,120.00,8040.0,14.925373,"[Financial, Rent To Own, Surveys, Sweepstakes]"
1,13125,13125-Auto Warranty-Admediary-Stateside Auto W...,Admediary,$24.00,CPL,Auto Warranty,NaN,120.00,7554.0,15.885624,"[Benefits, Credit Card, Financial, Personal Lo..."
2,13194,13194-Loan-AvenueLink-Funds-Now.com - AvenueLi...,AvenueLink,VARIABLE,RevShare,Loan,NaN,212.22,5270.0,40.269450,"[Credit Card, Financial, Personal Loan, Rent T..."
3,13221,13221-Credit Card-B2Direct-PlatinumCardApprova...,B2Direct,$1.50,CPA,Credit Card,NaN,124.50,5169.0,24.085897,"[Credit Card, Financial, Personal Loan, Survey..."
4,13089,13089-Home Insurance-B2Direct-HomeSafeGuarded ...,B2Direct,$12.00,CPA,Home Insurance,NaN,84.00,7122.0,11.794440,[Personal Loan]
5,13253,13253-Fast Card Finder - Bad Credit Credit Car...,PureAds,$2.00,CPL,Credit Card,None,85.50,4558.0,18.758227,"[Credit Card, Financial, Personal Loan, Survey..."


# Last 30 Days of Raw Data

In [24]:
#last 30 days of data minus current and last day

end_date = (dt.date.today() - dt.timedelta(days=1))  # Subtract today
start_date = end_date - dt.timedelta(days=30)

#convert date format
start_date= start_date.strftime('%Y-%m-%d')
end_date = end_date.strftime('%Y-%m-%d')

df_30 = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
df_30['Hitpath Offer ID'] = df_30['Hitpath Offer ID'].astype(str)
df_30['Hitpath Offer ID'] = df['Hitpath Offer ID'].replace(r'\.0$', '', regex=True)
df_30 = df_30[df_30['Send Strategy'] != 'AR']



<ipython-input-24-964be40463c5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_30['Hitpath Offer ID'] = df_30['Hitpath Offer ID'].astype(str)
<ipython-input-24-964be40463c5>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_30['Hitpath Offer ID'] = df['Hitpath Offer ID'].replace(r'\.0$', '', regex=True)


In [25]:
#dataset eCPM last 30 days
eCPM_df = df_30.groupby('shortcode_DP.SV').agg({'Revenue':'sum','Delivered':'sum'}).reset_index()
eCPM_df['eCPM'] = eCPM_df['Revenue'] * 1000 / eCPM_df['Delivered']

# Historical Offer Performance

In [26]:
#total historical offer performance
offer_performance = df.groupby(['shortcode_DP.SV', 'Offer', 'Hitpath Offer ID']).agg({'Revenue':'sum','Delivered':'sum', 'Opportunity Cost' : 'sum'}).reset_index()
offer_performance['Hitpath Offer ID'] = offer_performance['Hitpath Offer ID'].astype(int).astype(str)
offer_performance.rename(columns={'Revenue' : 'Overall Revenue',
                                 'Delivered': 'Overall Delivered'}, inplace=True)

#match shortcode_dp.sv
values_to_match = eCPM_df['shortcode_DP.SV']
offer_performance = offer_performance[offer_performance['shortcode_DP.SV'].isin(values_to_match)]
offer_performance['Overall Offer eCPM'] = offer_performance['Overall Revenue']*1000/offer_performance['Overall Delivered']

In [27]:
df[df['Hitpath Offer ID'] == 'OW13']

,Date,Scheduling Time,Offer,Hitpath Offer ID,DP.SV,Affiliate_Id,DP&Pub,Job_Id,Job_Name,Creative_Id,Creativename,Creative,Send Strategy,Shortcode,Start_Tstamp,Sent,Segments,Revenue,Conversions,Jump Page Clicks,Delivered,Not_Delivered,Optout,Clicks,Cost,Ecpm,Time,Publisher,Campaign,Route,Carrier,Dataset,Message,Responder Template,Keyword,Responder,Router Domain Name,c1,Responded,Response Rate,CTR,Gross Profit,Gross Margin,RPU,Provider,Code_Type,Revenue Source,Ar Day,Sub_Id,Campaign_Id,Roi,Shortcode Name,Total,Jump Page Version,Offer Wall Id,Offer Type,campaign_id,campaign_name,shortcode_DP.SV,Opportunity Cost Send Strategy,Dataset_Agg_30D_eCPM,Opportunity Cost,Offer Vertical,Offer Gap,Vertical Gap,eCPM ratio


In [28]:
#last date drop

# Convert the date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Group by the unique value(s) and find the index of the most recent date
idx = df.groupby(['shortcode_DP.SV', 'Offer', 'Hitpath Offer ID'])['Date'].idxmax()

# Use the index to select the rows with the most recent date
last_drop = df.loc[idx]
last_drop = last_drop[['Affiliate_Id','shortcode_DP.SV', 'Offer', 'Date', 'Delivered', 'Revenue']]
last_drop['Last Drop eCPM'] = last_drop['Revenue'] * 1000 / last_drop['Delivered']

# Merge the dataframes on the 'shortcode_DP.SV' and 'Offer' columns
offer_performance = pd.merge(offer_performance, last_drop, on=['shortcode_DP.SV', 'Offer'], how='left')
offer_performance = offer_performance.rename(columns={
    'Date': 'Last Drop Date',
})

offer_performance.rename(columns={'Delivered': 'Most Recent Delivered Volume', 
                                  'Revenue' : 'Most Recent Offer Revenue'}, inplace=True)

In [29]:
#check if the drop occured in the last 14 days

# Get the current date
current_date = pd.to_datetime('today')

# Calculate the difference in days between the current date and the 'Last Drop Date'
days_difference = (current_date - offer_performance['Last Drop Date']).dt.days

# Add a new column that tells if the date was within the last 14 days
offer_performance['Last 14 Days'] = days_difference <= 14

In [30]:
sorted_offer_performance = offer_performance.sort_values(by=['shortcode_DP.SV', 'Last Drop eCPM', 'Last 14 Days'], 
                                                    ascending=[True, False, True])

#merge statuses
sorted_offer_performance = sorted_offer_performance.merge(sms_offer[['Hitpath Offer ID', 'Status']], 
                                                          how='left', 
                                                          on='Hitpath Offer ID')

#select only production statuses
sorted_offer_performance = sorted_offer_performance[
    sorted_offer_performance['Status'] == 'Live']

In [31]:
#overall dataset eCPM
newdf = sorted_offer_performance.groupby('shortcode_DP.SV').agg({'Overall Revenue':'sum','Overall Delivered':'sum'}).reset_index()
newdf['Overall Dataset eCPM'] =  newdf['Overall Revenue']*1000/newdf['Overall Delivered']
newdf = newdf.drop(columns=['Overall Revenue', 'Overall Delivered'], axis=1)

In [32]:
#merge dataset eCPM
sorted_offer_performance = sorted_offer_performance.merge(newdf, on="shortcode_DP.SV", how="left")

#merge pubid restrictions
sorted_offer_performance = sorted_offer_performance.merge(sms_restrictions, on="Hitpath Offer ID", how="left")

In [33]:
#remove rows if aff id is in pubid restriction
sorted_offer_performance = sorted_offer_performance[~sorted_offer_performance.apply(lambda row: str(row['Affiliate_Id']) in str(row['Paused Pubs']), axis=1)]

#drop aff id and paused pub column
sorted_offer_performance = sorted_offer_performance.drop(columns=['Affiliate_Id','Paused Pubs'], axis = 1)

#drop mismatching ids
sorted_offer_performance['offer_abbreviation'] = sorted_offer_performance['Offer'].str[-3:]

# Extract the 3 letter shortcode abbreviation from the 'shortcode_DP.SV' column
sorted_offer_performance['shortcode_abbreviation'] = sorted_offer_performance['shortcode_DP.SV'].str.split('_').str[0]

# Filter rows where the abbreviations do not match
sorted_offer_performance = sorted_offer_performance[sorted_offer_performance['offer_abbreviation'] == sorted_offer_performance['shortcode_abbreviation']]

# Drop the temporary columns
sorted_offer_performance = sorted_offer_performance.drop(columns=['offer_abbreviation', 'shortcode_abbreviation'])

In [34]:
# #within 70% of overall
# sorted_offer_performance = sorted_offer_performance[sorted_offer_performance['Last Drop eCPM'] >= 0.7 * sorted_offer_performance['Overall Dataset eCPM']]


In [35]:
grouped = sorted_offer_performance.groupby('shortcode_DP.SV')
filtered_dfs = []

for name, group in grouped:
    # Filtering offers with Overall Offer eCPM >= Overall Dataset eCPM
    # filtered_group = group[group['Last Drop eCPM'] >= group['Overall Dataset eCPM']]
    filtered_group = group[group['Overall Offer eCPM'] >= group['Overall Dataset eCPM']]

    # If less than 7 offers, include offers with at least 90% of Overall Dataset eCPM, then 85%, and so on.
    thresholds = [0.9, 0.85, 0.8, 0.75, 0.7]
    for threshold in thresholds:
        if len(filtered_group) < 7:
#             additional_offers = group[group['Last Drop eCPM'] >= group['Overall Dataset eCPM'] * threshold]
            additional_offers = group[group['Overall Offer eCPM'] >= group['Overall Dataset eCPM'] * threshold]
            filtered_group = pd.concat([filtered_group, additional_offers]).drop_duplicates()
        else:
            break
   
    if len(filtered_group) > 7:
        filtered_group = filtered_group.sort_values(by='Overall Offer eCPM', ascending=False).head(7)

    filtered_dfs.append(filtered_group)

# Combining all filtered groups into a single DataFrame
filtered_df = pd.concat(filtered_dfs)

In [36]:
#Opp cost eCPM
filtered_df['Opportunity Cost eCPM'] = filtered_df['Opportunity Cost']*1000/filtered_df['Overall Delivered']
filtered_df = filtered_df.drop(columns=['Opportunity Cost'])

In [37]:
#Move Opporunity Cost eCPM Column
column_to_move = filtered_df.pop("Opportunity Cost eCPM")

# insert column with insert(location, column_name, column_value)

filtered_df.insert(5, "Opportunity Cost eCPM", column_to_move)

In [38]:
# BY DMA Account
filtered_df['extracted_DP.SV'] = filtered_df['shortcode_DP.SV'].str.split('_').str[-1]

#merge using 'extracted_DP.SV' and 'DP.SV'
filtered_df = pd.merge(filtered_df, pub_config[['DP.SV', 'DMA']], 
                     left_on='extracted_DP.SV', right_on='DP.SV', how='left')

#drop temp columns
filtered_df = filtered_df.drop(columns=['extracted_DP.SV', 'DP.SV'])

In [39]:
dma_separated_dfs = {}
for dma in filtered_df['DMA'].unique():
    if pd.notna(dma):  # Ensure that the 'DMA' value is not NaN
        dma_separated_dfs[dma] = filtered_df[filtered_df['DMA'] == dma]

# Create Data for Usage in Recommended Drops

In [40]:
#last 60 days of data minus current and last day

end_date = (dt.date.today() - dt.timedelta(days=1))  # Subtract today
start_date = end_date - dt.timedelta(days=30)

#convert date format
start_date= start_date.strftime('%Y-%m-%d')
end_date = end_date.strftime('%Y-%m-%d')

df_60 = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

In [41]:
sms_offer.columns

Index(['Hitpath Offer ID', 'Email Hitpath Offer ID', 'Scheduling Name',
       'Offer Name', 'Status', 'Status Changed Date', 'Delayed Reporting',
       'RX Rep', 'Advertiser', 'Data Partner Relationship',
       'Advertiser Offer ID', 'Direct Traffic Accepted ',
       'Additional Notes/Pre-pop Instructions', 'Vertical', 'Redirect Link',
       'Custom Creative Allowed', 'Custom Content Status',
       'Adv/Offer Abbreviation', 'Advertiser Rep', 'Scheduled Test Date',
       'Day Restrictions', 'Other Restrictions', 'Payout Change',
       'Payout Change Effective Date', '$ Payout', 'Payout Type',
       'Budget Status', 'Budget', 'Cap', 'Portal Acct Info', 'Live Pub IDs',
       'Paused Pubs', 'Special Pub Payouts', 'Pitch Approval',
       'Single/Multiple Pitch'],
      dtype='object')

In [42]:
# match dataset vertical to dp.sv
# df_dv_merged = df_60.merge(pub_config[['DP.SV', 'Sub Vertical', 'INTERNAL STATUS']], on='DP.SV', how='left')
df_dv_merged = df.merge(pub_config[['DP.SV', 'Sub Vertical', 'INTERNAL STATUS']], on='DP.SV', how='left')

# match offer vertical with hitpath ID
df_dvov_merged = df_dv_merged.merge(sms_offer[['Hitpath Offer ID', 'Vertical', 'Paused Pubs', 'Status']], on='Hitpath Offer ID', how='left')

#rename
df_dvov_merged.rename(columns={'Vertical': 'Offer Vertical', 'Sub Vertical' : 'Data Vertical'}, inplace=True)


In [43]:
#only approved offers
df_dvov_merged = df_dvov_merged[df_dvov_merged['Status'] == 'Live']

In [44]:
#active pubs
df_dvov_merged = df_dvov_merged[df_dvov_merged['INTERNAL STATUS'] == 'ACTIVE']

In [45]:
#delivered > 0
df_dvov_merged = df_dvov_merged[df_dvov_merged['Delivered'] > 0]


In [46]:
#remove lead group specific offers
a = ['12384', '12385', '12386']
df_dvov_merged = df_dvov_merged[~df_dvov_merged['Hitpath Offer ID'].isin(a)]


# Offer Vertical OCE By Data Vertical

In [47]:
df_dvov_merged = df_dvov_merged.loc[:,~df_dvov_merged.columns.duplicated()].copy()


In [48]:
#find data


data_offer_vert_OCE = df_dvov_merged.groupby(['shortcode_DP.SV', 'Offer Vertical']).agg({
    'Revenue':'sum',
    'Delivered':'sum', 
    'Opportunity Cost' : 'sum'}).reset_index()




In [49]:
df_dvov_merged

,Date,Scheduling Time,Offer,Hitpath Offer ID,DP.SV,Affiliate_Id,DP&Pub,Job_Id,Job_Name,Creative_Id,Creativename,Creative,Send Strategy,Shortcode,Start_Tstamp,Sent,Segments,Revenue,Conversions,Jump Page Clicks,Delivered,Not_Delivered,Optout,Clicks,Cost,Ecpm,Time,Publisher,Campaign,Route,Carrier,Dataset,Message,Responder Template,Keyword,Responder,Router Domain Name,c1,Responded,Response Rate,CTR,Gross Profit,Gross Margin,RPU,Provider,Code_Type,Revenue Source,Ar Day,Sub_Id,Campaign_Id,Roi,Shortcode Name,Total,Jump Page Version,Offer Wall Id,Offer Type,campaign_id,campaign_name,shortcode_DP.SV,Opportunity Cost Send Strategy,Dataset_Agg_30D_eCPM,Opportunity Cost,Offer Vertical,Offer Gap,Vertical Gap,eCPM ratio,Data Vertical,INTERNAL STATUS,Paused Pubs,Status
1,2024-05-15,2024-05-15 10:00,12089 - BE DS HZB,12089,CM.OSR,461452,CM.OSR_461452,664438.0,SS_HZB_CM-OSR-30DC_12089_P_15May24,804423.0,12089.SC.HZB.453342,HZB: These schools might be interested in enro...,P,44345.0,2024-05-15 13:02:08,NaN,HZB_CM.OSR_30DC,68.000,2.0,9.0,2744.0,NaN,33.000000,35.0,19.208,24.781341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Short Code,Short Code - SS Jobs,NaN,NaN,NaN,-29.350,HZB,NaN,0.0,NaN,Single Offer,12089.0,Degree Snap - SMS (Private for SID 3529) - Ban...,HZB_CM.OSR,True,12.505654,33.684484,Education,NaN,57.0,1.981611,Surveys,ACTIVE,NaN,Live
2,2024-05-15,2024-05-15 10:00,13113 - HZB,13113,AL.PL.3,461838,AL.PL.3_461838,664449.0,SS_HZB_AL-PL-3-30DC-VZN_13113_P_15May24,865005.0,13113.SC.HZB.453641,HZB: You may qualify for a reduced auto insura...,P,44345.0,2024-05-15 13:01:28,NaN,HZB_AL.PL.3_30DC_VZN,9.828,1.0,3.0,820.0,NaN,13.000000,7.0,5.740,11.985366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Short Code,Short Code - SS Jobs,NaN,NaN,NaN,1.742,HZB,NaN,0.0,NaN,Single Offer,NaN,NaN,HZB_AL.PL.3,True,30.541005,-15.215624,Auto Insurance,13.0,13.0,0.392435,Personal Loan,ACTIVE,NaN,Live
3,2024-05-15,2024-05-15 10:00,13194 - HZB,13194,B2.F,461123,B2.F_461123,664676.0,SS_HZB_B2-F-30DC-VZN_13194_OT_15May24,890528.0,13194.SC.HZB.453941,"HZB: {{coalesce|first_name|Today}}, we're issu...",OT,44345.0,2024-05-15 13:01:08,NaN,HZB_B2.F_30DC_VZN,0.000,0.0,76.0,1508.0,NaN,8.000000,150.0,10.556,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Short Code,Short Code - SS Jobs,NaN,NaN,NaN,10.630,HZB,NaN,0.0,NaN,Single Offer,NaN,NaN,HZB_B2.F,True,14.095742,-21.256379,Loan,NaN,NaN,0.000000,Financial,ACTIVE,NaN,Live
4,2024-05-15,2024-05-15 10:00,11600 - WI UG HZB,11600,DOT.PL,461039,DOT.PL_461039,664509.0,SS_HZB_DOT-PL-30DC-VZN_11600_P_15May24,868184.0,11600.SC.HZB.453901,HZB: Hey {{coalesce|first_name|there}}! Here's...,P,44345.0,2024-05-15 13:02:08,NaN,HZB_DOT.PL_30DC_VZN,7.700,14.0,28.0,1229.0,NaN,12.000000,51.0,8.603,6.265256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Short Code,Short Code - SS Jobs,NaN,NaN,NaN,0.990,HZB,NaN,0.0,NaN,Single Offer,11600.0,Unemployments Benefits Guide - Stimulus Carous...,HZB_DOT.PL,True,11.127660,-5.975895,Resources,7.0,7.0,0.563034,Personal Loan,ACTIVE,NaN,Live
5,2024-05-15,2024-05-15 10:00,12640 - PS FC HZB,12640,AMD.PL,461810,AMD.PL_461810,664447.0,SS_HZB_AMD-PL-30DC-VZN_12640_P_15May24,868105.0,12640.SC.HZB.453841,"HZB: {{coalesce|first_name|Hey}}, check to see...",P,44345.0,2024-05-15 13:01:37,NaN,HZB_AMD.PL_30DC_VZN,21.000,7.0,55.0,1832.0,NaN,3.000000,108.0,12.824,11.462882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Short Code,Short Code - SS Jobs,NaN,NaN,NaN,-8.150,HZB,NaN,0.0,NaN,Single Offer,12640.0,Found Cash - Pushnami (SMS),HZB_AMD.PL,True,11.602864,-0.256447,Resources,8.0,8.0,0.987936,Personal Loan,ACTIVE,NaN,Live
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [50]:
#calculate Opportunity Cost eCPM
data_offer_vert_OCE['Opportunity Cost eCPM'] = data_offer_vert_OCE['Opportunity Cost'
                                                                  ]*1000/data_offer_vert_OCE['Delivered']

#calculate eCPM
data_offer_vert_OCE['eCPM'] = data_offer_vert_OCE['Revenue']*1000/data_offer_vert_OCE['Delivered']


In [51]:
data_offer_vert_OCE[data_offer_vert_OCE['Opportunity Cost eCPM'] > -3]

,shortcode_DP.SV,Offer Vertical,Revenue,Delivered,Opportunity Cost,Opportunity Cost eCPM,eCPM
2,A4FLC_I.A4F,Loan,3.1800,603.0,-1.391129,-2.307014,5.273632
4,CSS_AI.CC,Credit Score,100.8000,9973.0,-13.855644,-1.389316,10.107290
5,CSS_AI.CC,Education,262.0000,19796.0,10.216840,0.516106,13.234997
6,CSS_AI.CC,Resources,88.5500,7115.0,-3.838597,-0.539508,12.445538
7,CSS_EDM.247L,Credit Card,4763.0000,367286.0,376.391586,1.024792,12.968096
11,CSS_FSM.YS,Credit Card,1507.0000,162429.0,-265.048454,-1.631780,9.277900
12,CSS_FSM.YS,Credit Score,201.6000,10382.0,101.766084,9.802166,19.418224
16,CSS_I.CC,Credit Card,1346.0000,61214.0,-21.244051,-0.347046,21.988434
18,CSS_I.CC,Education,322.0000,11674.0,9.362572,0.802002,27.582662
20,CSS_NPD.RTO,Credit Card,4121.5000,485099.0,-779.303407,-1.606483,8.496204


In [52]:
#filter if greater than -1
data_offer_vert_OCE = data_offer_vert_OCE[data_offer_vert_OCE['Opportunity Cost eCPM'] > -3]
data_offer_vert_OCE = data_offer_vert_OCE[data_offer_vert_OCE['eCPM'] > 7]


# Offer Opportunity Cost eCPM

In [53]:
#find data 
offer_OCE = df_dvov_merged.groupby(['Hitpath Offer ID','Offer Vertical']).agg({
    'Revenue':'sum',
    'Delivered':'sum', 
    'Opportunity Cost' : 'sum'}).reset_index()


In [54]:
#calculate Opportunity Cost eCPM
offer_OCE['Opportunity Cost eCPM'] = offer_OCE['Opportunity Cost']*1000/offer_OCE['Delivered']
offer_OCE['eCPM'] = offer_OCE['Revenue']*1000/offer_OCE['Delivered']


In [55]:
#filter if greater than -1
offer_OCE = offer_OCE[offer_OCE['Opportunity Cost eCPM'] > -5]
offer_OCE = offer_OCE[offer_OCE['eCPM'] > 7]



# Rec

In [56]:
#remove old shortcode_DP.SV
# Combine 'DP.SV' and 'shortcode_DP.SV' to create a unique key
df_dvov_merged['unique_key'] = df_dvov_merged['DP.SV'] + "_" + df_dvov_merged['shortcode_DP.SV']

# Find the latest date for each unique key
latest_dates = df_dvov_merged.groupby('unique_key')['Date'].max()

# Calculate the date 30 days ago from the most recent date in the dataset
date_threshold = df_dvov_merged['Date'].max() - dt.timedelta(days=15)

# Identifiers that have had a drop within the last 30 days
active_keys = latest_dates[latest_dates > date_threshold]

# Filter the original DataFrame to only include rows with these active keys
df_dvov_merged = df_dvov_merged[df_dvov_merged['unique_key'].isin(active_keys.index)]



In [57]:
# Create an empty dataframe for the updated recommendations
recommendations = pd.DataFrame(columns=["shortcode_DP.SV", "PubID", 
                                               "Hitpath Offer ID","Offer Vertical"])

# Iterate over each unique shortcode_DP.SV in df
for shortcode in df_dvov_merged['shortcode_DP.SV'].unique():
    # Filter the dataframe for the specific shortcode
    temp_df = df_dvov_merged[df_dvov_merged['shortcode_DP.SV'] == shortcode]
    
    # Extract the data vertical and PubID
    shortcode_DPSV = temp_df['shortcode_DP.SV'].iloc[0]
    pub_id = temp_df['Affiliate_Id'].iloc[0]
    
    # Iterate over each Hitpath ID in offer_OCE
    for _, row in offer_OCE.iterrows():
        hitpath_id = row['Hitpath Offer ID']
        offer_vertical = row['Offer Vertical']
        
        # Check if this Hitpath ID has been sent by the current shortcode_DP.SV
        if hitpath_id not in temp_df['Hitpath Offer ID'].values:
            # Check if the shortcode_DP.SV's Data Vertical appears in offer_OCE for this Offer Vertical
            potential_matches = data_offer_vert_OCE[(data_offer_vert_OCE['shortcode_DP.SV'] == shortcode_DPSV) & 
                                             (data_offer_vert_OCE['Offer Vertical'] == offer_vertical)]
            if len(potential_matches) > 0:
                # Check if the PubID is not in the Paused Pubs column for this Hitpath ID
                paused_pubs = df_dvov_merged[df_dvov_merged['Hitpath Offer ID'] == hitpath_id]['Paused Pubs'].values
                if len(paused_pubs) > 0 and pub_id not in paused_pubs:
                    recommendations = recommendations.append({
                        "shortcode_DP.SV": shortcode,
                        "PubID": pub_id,
                        "Hitpath Offer ID": hitpath_id,
                        "Offer Vertical": offer_vertical
                    }, ignore_index=True)

recommendations

,shortcode_DP.SV,PubID,Hitpath Offer ID,Offer Vertical
0,HZB_AL.PL.3,461838,12305,Loan
1,HZB_AL.PL.3,461838,12306,Loan
2,HZB_AL.PL.3,461838,13073,Loan
3,HZB_AL.PL.3,461838,13074,Loan
4,HZB_DOT.PL,461039,12089,Education
5,HZB_DOT.PL,461039,12305,Loan
6,HZB_DOT.PL,461039,12306,Loan
7,HZB_DOT.PL,461039,12342,Loan
8,HZB_DOT.PL,461039,12611,Loan
9,HZB_DOT.PL,461039,13073,Loan


In [58]:
#checking for paused pubids
recommendations = recommendations.merge(sms_offer[['Hitpath Offer ID', 'Scheduling Name']], on='Hitpath Offer ID', how='left')
recommendations = recommendations.merge(sms_restrictions, on="Hitpath Offer ID", how="left")

In [59]:
#remove rows if aff id is in pubid restriction
recommendations = recommendations[~recommendations.apply(lambda row: str(row['PubID']) in str(row['Paused Pubs']), axis=1)]

#drop aff id and paused pub column
recommendations = recommendations.drop(columns=['PubID','Paused Pubs'], axis = 1)

In [60]:
# mamba_future['Hitpath Offer ID'] = mamba_future['Hitpath Offer ID'].astype(int).astype(str)

#matching hitpath id and shortcode to find matches in rec tab to future scheduledd
matches = recommendations.apply(
    lambda x: any(
        (mamba_future['Hitpath Offer ID'] == x['Hitpath Offer ID']) & 
        (mamba_future['shortcode_DP.SV'] == x['shortcode_DP.SV'])
    ), 
    axis=1
)

# remove matching rows from recommendations tab
recommendations_final = recommendations[~matches]

recommendations_final

,shortcode_DP.SV,Hitpath Offer ID,Offer Vertical,Scheduling Name
0,HZB_AL.PL.3,12305,Loan,12305-Loan-The Lead Group-Market Loans - Perso...
1,HZB_AL.PL.3,12306,Loan,12306-Loan-The Lead Group-100 Lenders Personal...
2,HZB_AL.PL.3,13073,Loan,13073-Loan-Leap Theory-Coastal City Loans - Le...
3,HZB_AL.PL.3,13074,Loan,13074-Loan-Leap Theory-Loans Prosper - Leap Th...
4,HZB_DOT.PL,12089,Education,12089-Education-Banner Edge-Degree Snap - SMS ...
5,HZB_DOT.PL,12305,Loan,12305-Loan-The Lead Group-Market Loans - Perso...
6,HZB_DOT.PL,12306,Loan,12306-Loan-The Lead Group-100 Lenders Personal...
7,HZB_DOT.PL,12342,Loan,12342-Loan-The Lead Group-PickALender.com Pers...
8,HZB_DOT.PL,12611,Loan,12611-Loan-AvenueLink-QuickLoans.cash - Avenue...
9,HZB_DOT.PL,13073,Loan,13073-Loan-Leap Theory-Coastal City Loans - Le...


In [61]:
agg = offer_performance.groupby('Hitpath Offer ID').agg({
    'Overall Revenue': 'sum',
    'Overall Delivered': 'sum'
}).reset_index()

# Calculating eCPM for each aggregated offer ID
agg['eCPM'] = (agg['Overall Revenue'] / agg['Overall Delivered']) * 1000


In [62]:
merged = recommendations_final.merge(agg, left_on='Hitpath Offer ID', right_on='Hitpath Offer ID', how='left')

merged = merged[['shortcode_DP.SV', 'Hitpath Offer ID', 'Offer Vertical', 'Scheduling Name', 'eCPM']]


In [63]:
sorted_data = merged.groupby('shortcode_DP.SV').apply(lambda x: x.sort_values('eCPM', ascending=False)).reset_index(drop=True)


In [64]:
#output file
filename = filepath.output_folder+'/Redrop - {}.xlsx'.format(dt.date.today().strftime("%m_%d_%Y"))

#create writer
writer = pd.ExcelWriter(filename, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
with pd.ExcelWriter(filename) as writer:
    for dma, df in dma_separated_dfs.items():
        sheet_name = f'{dma} - Redrop'  # dynamic sheet name based on the DMA
        df.to_excel(writer, sheet_name=sheet_name, index=False)  # Writing data to different sheets

with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer:
    sorted_data.to_excel(writer, sheet_name='Offer Recommendations', index=False)

with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer:
    passed_offers.to_excel(writer, sheet_name='Passed Offers - Last 30 days', index=False)

# Close the Pandas Excel writer and output the Excel file.
# writer.close()

In [65]:
toaddr = ['lili@rxmg.com','nathan@rxmg.com', 'prasad@rxmg.com', 'nina@rxmg.com']
# toaddr = ['nathan@rxmg.com']
today = dt.date.today().strftime("%m_%d_%Y")
subject_line = f"SMS Team Weekly Redrop/Recommendations Report - {today}"
email_body = "Hi Team, here is the Weekly Redrop Report \n\n"

for i in toaddr:
    send_email.send_email([filename],subject_line,email_body,i)
 